# EXAMPLE 1 
## Explaining the experiment
In this example, we are investigating a 2Mx8 SRAM that was exposed to radiation in pseudostatic tests. Hence, the content inside the memory was written, left in standby for 30s, read and corrected. This step was repeated some tens of times. This process was done for three different patterns (0x00, 0x55, 0xFF) yielding three different CSV files, which can be found in this folder. 

## Loading packages
The very first thing we must do is to load the packages required to load files (_DelimitedFiles_) as well as the LELAPE module. I suppose you have installed both. Load is done with:

In [ ]:
using DelimitedFiles, LELAPE

## Defining variables
Previous paragraph allows us to define several variables for checking the tests:

* Word width : 8 bits
* Memory size in words: 2M is just 2^21.
* In SRAMs, it seems more likely to succeed the XOR operation.
* Tests were pseudostatic. Therefore, it is intelligent to keep information about the different cycles.

Ok, let us use this information to set these variables:

In [ ]:
LA = 2^21 # Memory size in words
WordWidth = 8 # Selfexplaining.
Operation = "XOR" # Only "XOR" or "POS" are allowed.
KeepCycles = true # This is a Bool variable and only true false are accepted.

## Loading data
Results are stored in three different files following the required format: 
* CSV files 
* Every row is formed as WORD ADDRESS, READ VALUE, PATTERN, CYCLE.
Besides, the first row contains column heading (must be skipped), separators are commas and EOL character is the standard. 

We will use the _readdlm_ function provided by the _DelimitedFiles_ package to load the first CSV file and to store everything in the new variable, DATA. Finally, it is important to indicate that DATA must be an array of UInt32 numbers. 

In [ ]:
DATA1 = readdlm("ExampleSRAM01.csv", ',', UInt32, '\n', skipstart=1)

Good!! If you have correctly proceeded, a 115x4 unsigned integer matrix is loaded. Now, let us analyze the DATA but we need to define in advance several variables to set the analysis.

## Looking for MBUs
This analyisis is quite simple. We will call the _CheckMBUs_ functionthat returns the MBUs present in DATA.Input arguments are the second and third columns, and the wordwidth.

This function returns two vectors. The first one indicates in position _k_ the number of bitflips observed in the _kth_ word. The second one is a vector of vectors and contains more detailed information: not only the number of bitflips per word but the position of the flipped bit (0 = LSB, WordWidth-1 = MSB). 

In [ ]:
MBUSize, MBU_bit_pos = CheckMBUs(DATA1[:,2], DATA1[:,3], WordWidth)

The following loop will show how many MBUs per number of flipped bits were observed:

In [ ]:
for size = 1: WordWidth
    println("$size-bit MBUs: ", length(findall(MBUSize.==size)))
end

## Looking for MCUs
As modern memories are interleaved, it is not worth investigating MBUs but MCUs. Now, the system will combine addresses in all the possible pairs and operate them to create a DV set. If there were no MCUs, their characteristics are known. 

In particular, we can state that if the expected number of elements repeated _k_ times in this set is lower than a very low positive number, it is impossible to observe this number of repetitions unless the Only SBU assumption fails. We will define this threshold as 0.001 (default, 0.05). 


Although without a solid theoretical background, it seems that using pseudoaddress instead of word address provides better results.

Some experiments seem to show that if an element with very few number of 1s in binary format is too often repeated, it is indicative of the presence of MCUs. This is the Trace Rule and, in our analyisis, we want to keep all those too often repeated elements such that contain 2 ones or less in binary format.

Finally, perhaps we know that MCUs will not very large. For example, we may guess that MCUs with more than 20 bitflips are totally rejected. Therefore, to help the software and to avoid running out of memory, we will say the program _"Don't be silly and do not expect events larger than 20!!"_ If somehow this idea was wrong, we can change this value again and repeat the calculations.

In [ ]:
ϵ = 0.001   # If the expected number of elements repeated k times is lower than ϵ, 
            # we can afirm that this is virtually impossible.
UsePseudoAddress = true
TraceRuleLength = 2
LargestMCUSize = 20

Time to test!!! We will call the function. Deppending on the set size or even if this is your first test, it will take you more or less time (Don't get up from your chair, though!!!!)

The following instruction will look for:
1. Values that pass the self-consistency test (C1_SCY)
2. Values found after inspecting MCUs derived from self-consistency-test (C1_MCU).
3. Values with less than or equal to _TraceRuleLength_ 1s in binary format that appear too often in the DV set (C1_TRC).
4. Values that, after combining in pairs the union of all the previous three sets and applying the operation and that appear too many times within the DV set (C1_SHF).

The first column of each matrix are the possible values and the second one the times it appeared.

In [ ]:
C1_SCY, C1_MCU, C1_TRC, C1_SHF = DetectAnomalies_FullCheck(DATA1, WordWidth, LA, Operation, TraceRuleLength, UsePseudoAddress, KeepCycles, ϵ, LargestMCUSize)


Perhaps these matrices are hard to read since, for efficiency, they were returned in UInt32 format, even the number of occurrences!!! Execute the following instrucction for a better comprehension.

In [ ]:
println("Elements appearing more than expected and passing the Self-Consistency test:\n")
for index in 1:length(C1_SCY[:, 1])
    println("Value: 0x", string(C1_SCY[index, 1], base=16, pad = 6), " --> ", Int(C1_SCY[index, 2]),".")
end

UsePseudoAddress ? L = LA*WordWidth : L = LA

print("\nOnly up to ", MaxExpectedRepetitions(NPairs(DATA1, UsePseudoAddress, WordWidth, KeepCycles), L, Operation, ϵ)-1, " repetitions are explained by randomness.")

In this example, it is not worth to check the other sets since they did not yield any positive result. If you had had success, you would only have to do the following:

In [ ]:
C1_All = [C1_SCY; C1_MCU; C1_TRC; C1_SHF]

## Grouping bitflips
Now, we have discovered those values relating pairs of pseudoaddresses. Now, let us go to group events in DATA. 

The first step consists in labeling all the pseudoaddresses and grouping their assigned indexes to a matrix containing information for the possible MCUs. It is an intermediate step and is done with the instruction _MCU_Indexes_ with the required and already defined parameters. 

In [ ]:
Labeled_addresses = MCU_Indexes(DATA1, Operation, C1_All[:, 1], UsePseudoAddress, WordWidth)

You should have got a 19x4 matrix so there are 19 MCUs and the largest one affects 4 bits. For example, the first row should be 6, 7, 8 and 9. This means that bitflips in these rows in DATA belong to the same bitflip. Be careful!! We can do that since there are not MBUs and index for word addresses and pseudoaddresses are identical. 

 However, we need perhaps something else: separate MCUS per size. To do that, we use a final function, _Classify_Addresses_in_MCU()_, using the previous matrix, _Labeled_addresses_. This function returns a vector of matrices. The first element is a matrix with pseudoaddresses related to 4-bit MCUs, the second to 3-bit MCUs, the third to 2-bit MCUs, and the fourth and last one to SBUs.

In [ ]:
Events = Classify_Addresses_in_MCU(DATA1, Labeled_addresses, UsePseudoAddress, WordWidth)

---------------------
Difficult to read, isn't it? The following instruction makes the content more readable:

In [ ]:
for k = 1:length(Events) 
    NMCUs = length(Events[k][:, 1])
    println("Pseudoaddresses involved in $(length(Events)-k+1)-bit MCUs ($NMCUs events):")
    for row = 1:NMCUs
        for bit = 1:length(Events)-k+1
            print("0x", string(Events[k][row, bit], base=16, pad = 6), )
            
            bit != length(Events)-k+1 ? print(", ") : print("\n")

        end
    end
    println()
end

## Analysis completed!
However, perhaps you have not forgotten that there are other 2 sets of data in this folder, got from testing the memory with other patterns. The following cells will discover the anomalously repeated DV elements. It has been observed that these elements strongly deppend on the pattern value. 

Let us go with the __Second Set__:

In [ ]:
DATA2 = readdlm("ExampleSRAM02.csv", ',', UInt32, '\n', skipstart=1);
C2_SCY, C2_MCU, C2_TRC, C2_SHF = DetectAnomalies_FullCheck(DATA2, WordWidth, LA, Operation, TraceRuleLength, UsePseudoAddress, KeepCycles, ϵ, LargestMCUSize)
C2_All = [C2_SCY; C2_MCU; C2_TRC; C2_SHF]

println("Elements appearing more than expected and passing the Self-Consistency and rest of tests:\n")
for index in 1:length(C2_All[:, 1])
    println("Value: 0x", string(C2_All[index, 1], base=16, pad = 6), " --> ", Int(C2_All[index, 2]),".")
end

UsePseudoAddress ? L = LA*WordWidth : L = LA

print("\nOnly up to ", MaxExpectedRepetitions(NPairs(DATA2, UsePseudoAddress, WordWidth, KeepCycles), L, Operation, ϵ)-1, " repetitions are explained by randomness.")

You can see that other critical DV elements have appeared. We are gathering more and more information before the final analysis.

Now, the __Third Set__:

In [ ]:
DATA3 = readdlm("ExampleSRAM03.csv", ',', UInt32, '\n', skipstart=1);
C3_SCY, C3_MCU, C3_TRC, C3_SHF = DetectAnomalies_FullCheck(DATA3, WordWidth, LA, Operation, TraceRuleLength, UsePseudoAddress, KeepCycles, ϵ, LargestMCUSize)
C3_All = [C3_SCY; C3_MCU; C3_TRC; C3_SHF]

println("Elements appearing more than expected and passing the Self-Consistency and rest of tests:\n")
for index in 1:length(C3_All[:, 1])
    println("Value: 0x", string(C3_All[index, 1], base=16, pad = 6), " --> ", Int(C3_All[index, 2]),".")
end

UsePseudoAddress ? L = LA*WordWidth : L = LA

print("\nOnly up to ", MaxExpectedRepetitions(NPairs(DATA3, UsePseudoAddress, WordWidth, KeepCycles), L, Operation, ϵ)-1, " repetitions are explained by randomness.")

## Reanalyzing Data
Making different tests has allowed to collect a lot of information about the elements that relate pairs of pseudoaddresses. Now, let us combine the specific values into a complete set and reanalyze elements. 

Thus, the whole set of discovered values is:

In [ ]:
C123_All = sort(union(C1_All[:,1], C2_All[:,1], C3_All[:, 1]))

println("Critical DV elements relating pairs of pseudoaddresses after XORing:\n")
for index in 1:length(C123_All[:, 1])
    println("Value: 0x", string(C123_All[index, 1], base=16, pad = 6))
end

println("\nUuuups, ", length(C123_All), " elements to try to look for MCUs!")

OK. Now, reanalyse all the three set of DATA with the previous new values.

__First SET:__

In [ ]:
Labeled_addresses = MCU_Indexes(DATA1, Operation, C123_All[:, 1], UsePseudoAddress, WordWidth)
Events = Classify_Addresses_in_MCU(DATA1, Labeled_addresses, UsePseudoAddress, WordWidth)
for k = 1:length(Events) 
    NMCUs = length(Events[k][:, 1])
    println("Pseudoaddresses involved in $(length(Events)-k+1)-bit MCUs ($NMCUs events):")
    for row = 1:NMCUs
        for bit = 1:length(Events)-k+1
            print("0x", string(Events[k][row, bit], base=16, pad = 6), )
            
            bit != length(Events)-k+1 ? print(", ") : print("\n")

        end
    end
    println()
end

You can realize that there are slight differences with the initial analysis: a new 3-bit MCU appeared as the combination of one 2-bit MCU and an SBU.

Let us analyze the __SECOND SET__.

In [ ]:
Labeled_addresses = MCU_Indexes(DATA2, Operation, C123_All[:, 1], UsePseudoAddress, WordWidth)
Events = Classify_Addresses_in_MCU(DATA2, Labeled_addresses, UsePseudoAddress, WordWidth)
for k = 1:length(Events) 
    NMCUs = length(Events[k][:, 1])
    println("Pseudoaddresses involved in $(length(Events)-k+1)-bit MCUs ($NMCUs events):")
    for row = 1:NMCUs
        for bit = 1:length(Events)-k+1
            print("0x", string(Events[k][row, bit], base=16, pad = 6), )
            
            bit != length(Events)-k+1 ? print(", ") : print("\n")

        end
    end
    println()
end

And the __THIRD SET__:

In [ ]:
Labeled_addresses = MCU_Indexes(DATA3, Operation, C123_All[:, 1], UsePseudoAddress, WordWidth)
Events = Classify_Addresses_in_MCU(DATA3, Labeled_addresses, UsePseudoAddress, WordWidth)
for k = 1:length(Events) 
    NMCUs = length(Events[k][:, 1])
    println("Pseudoaddresses involved in $(length(Events)-k+1)-bit MCUs ($NMCUs events):")
    for row = 1:NMCUs
        for bit = 1:length(Events)-k+1
            print("0x", string(Events[k][row, bit], base=16, pad = 6), )
            
            bit != length(Events)-k+1 ? print(", ") : print("\n")

        end
    end
    println()
end

Now, you can go on analyzing your data at your will. For example, you can find out cross sections, ratios between events, etc. 

## What else can I do?
We advice you to save a copy of this notebook somewhere and start to modify conditions. For example, you can:

* change the operation into "POS"
* use Word Address instead of the Pseudoaddress
* suppose that all the events were got after only 1 cycle

You must go back to previous cells and change the different variables: Operation, UsePseudoADD or KeepCycles.

Enjoy it!